In [29]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from joblib import dump
 


# Загружаем данные из CSV-файла
data = pd.read_csv('cirrhosis.csv')

# Посмотрим на структуру данных и первые строки
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             418 non-null    int64  
 1   N_Days         418 non-null    int64  
 2   Status         418 non-null    object 
 3   Drug           312 non-null    object 
 4   Age            418 non-null    int64  
 5   Sex            418 non-null    object 
 6   Ascites        312 non-null    object 
 7   Hepatomegaly   312 non-null    object 
 8   Spiders        312 non-null    object 
 9   Edema          418 non-null    object 
 10  Bilirubin      418 non-null    float64
 11  Cholesterol    284 non-null    float64
 12  Albumin        418 non-null    float64
 13  Copper         310 non-null    float64
 14  Alk_Phos       312 non-null    float64
 15  SGOT           312 non-null    float64
 16  Tryglicerides  282 non-null    float64
 17  Platelets      407 non-null    float64
 18  Prothrombi

<!-- ID - уникальный идентификатор
N_Days - количество дней между регистрацией и более ранним моментом смерти, трансплантации или анализа исследования в июле 1986 г.
Status - статус пациента C (подвергнуто цензуре), CL (подвергнуто цензуре из-за повреждения печени), D (смерть)
Drug - тип лекарственного средства D-пеницилламин или плацебо
Age - возраст (в днях)
Sex - M (мужчина), F (женщина)
Ascites - наличие асцита N (нет), Y (Да)
Hepatomegaly - наличие гепатомегалии N (нет), Y (да)
Spiders - наличие пауков N (нет), Y (да)
Edema - наличие отека N (отсутствие отека и отсутствие терапии диуретиками при отеке), S (отек присутствует без диуретиков или отек устраняется диуретиками), Y (отек, несмотря на терапию диуретиками)
Bilirubin - сывороточный билирубин 
Cholesterol - сывороточный холестерин 
Albumin - альбумин )
Copper - медь в моче (мкг/день)
Alk_Phos - aщелочная фосфатаза (Ед/л)
SGOT - SGOT (Ед/мл)
Triglycerides - триглицериды
Platelets - количество тромбоцитов на куб (мл/1000)
Prothrombin - протромбиновое время (сек)
Stage - гистологическая стадия заболевания (1 2 3 4) -->


In [2]:
data[:5]

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,1,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,2,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,3,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,4,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,5,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0


In [3]:
data.isna().sum()

ID                 0
N_Days             0
Status             0
Drug             106
Age                0
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol      134
Albumin            0
Copper           108
Alk_Phos         106
SGOT             106
Tryglicerides    136
Platelets         11
Prothrombin        2
Stage              6
dtype: int64

In [4]:
# воспользуемся классом SimpleImputer для восстановления пропущенных значений (https://habr.com/ru/companies/otus/articles/681410/)
si = SimpleImputer(missing_values=np.nan,strategy="mean")
si.fit(data.iloc[:,10:-1])

data.iloc[:,10:-1] = si.transform(data.iloc[:,10:-1])

In [5]:
data.isnull().sum()

ID                 0
N_Days             0
Status             0
Drug             106
Age                0
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol        0
Albumin            0
Copper             0
Alk_Phos           0
SGOT               0
Tryglicerides      0
Platelets          0
Prothrombin        0
Stage              6
dtype: int64

In [6]:
# так как остались пропуски в количестве 106, удалим их с помощью dropna
data = data.dropna(subset=['Ascites'])

In [7]:
data.isnull().sum()

ID               0
N_Days           0
Status           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
dtype: int64

In [8]:
# конвертируем возраст пациентов в годы
data["Age"] = (data["Age"].values/365).round()
data[:5]

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,1,400,D,D-penicillamine,59.0,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,2,4500,C,D-penicillamine,56.0,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,3,1012,D,D-penicillamine,70.0,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,4,1925,D,D-penicillamine,55.0,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,5,1504,CL,Placebo,38.0,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0


In [9]:
# заменим статусы на числовые значения
statuses = list(data["Status"])
status_num = []
for i in range(len(statuses)):
    if statuses[i]=="D":
        status_num.append(0)
    elif statuses[i]=="C":
        status_num.append(1)
    else :
        status_num.append(2)

In [10]:
data = data.drop(["Status"], axis=1)
data["status"] = status_num
data[:5]

,ID,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,status
0,1,400,D-penicillamine,59.0,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0,0
1,2,4500,D-penicillamine,56.0,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0,1
2,3,1012,D-penicillamine,70.0,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0,0
3,4,1925,D-penicillamine,55.0,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0,0
4,5,1504,Placebo,38.0,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0,2


In [11]:
# разделим данные и выберем колонки для фичей
X = data.iloc[:, 1:-1].values
# выберем последнюю колонку в качестве таргета
y = data.iloc[:, -1].values

In [12]:
X[:5]

array([[400, 'D-penicillamine', 59.0, 'F', 'Y', 'Y', 'Y', 'Y', 14.5,
        261.0, 2.6, 156.0, 1718.0, 137.95, 172.0, 190.0, 12.2, 4.0],
       [4500, 'D-penicillamine', 56.0, 'F', 'N', 'Y', 'Y', 'N', 1.1,
        302.0, 4.14, 54.0, 7394.8, 113.52, 88.0, 221.0, 10.6, 3.0],
       [1012, 'D-penicillamine', 70.0, 'M', 'N', 'N', 'N', 'S', 1.4,
        176.0, 3.48, 210.0, 516.0, 96.1, 55.0, 151.0, 12.0, 4.0],
       [1925, 'D-penicillamine', 55.0, 'F', 'N', 'Y', 'Y', 'S', 1.8,
        244.0, 2.54, 64.0, 6121.8, 60.63, 92.0, 183.0, 10.3, 4.0],
       [1504, 'Placebo', 38.0, 'F', 'N', 'Y', 'Y', 'N', 3.4, 279.0, 3.53,
        143.0, 671.0, 113.15, 72.0, 136.0, 10.9, 3.0]], dtype=object)

In [13]:
y[:5]

array([0, 1, 0, 0, 2], dtype=int64)

In [14]:
# фичи в колонках 1 3 4 5 6 7 представленны текстовыми данными, преобразуем их в числовые
# https://www.geeksforgeeks.org/prediction-using-columntransformer-onehotencoder-and-pipeline/

ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1, 3, 4, 5, 6, 7])], remainder="passthrough" )
X = np.array(ct.fit_transform(X))

In [15]:
# train/test 0.8/0.2  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2)

In [16]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rfmean = cross_val_score(estimator=rf, X=X_train, y=y_train, cv =10)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

In [23]:
gb = GradientBoostingRegressor()
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
gbmean = cross_val_score(estimator=gb, X=X_train, y=y_train, cv =10)
mae_gb = mean_absolute_error(y_test, y_pred_gb)

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
lrmean = cross_val_score(estimator=lr, X=X_train, y=y_train, cv =10)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

In [25]:
print(f"RandomForest mae: {mae_rf}")
print(f"RandomForest Accuracy: {rfmean.mean()*100} %\n")

RandomForest mae: 0.40333333333333343
RandomForest Accuracy: 25.648821376770055 %



In [26]:
print(f"GradientBoosting mae: {mae_gb}")
print(f"GradientBoosting Accuracy: {gbmean.mean()*100} %\n")

GradientBoosting mae: 0.3920210574364829
GradientBoosting Accuracy: 25.187009578191955 %



In [27]:
print(f"GradientBoosting mae: {mae_lr}")
print(f"GradientBoosting Accuracy: {lrmean.mean()*100} %\n")

GradientBoosting mae: 0.4319441558412182
GradientBoosting Accuracy: 18.74177332440196 %



In [30]:
dump(rf, 'rf.joblib')
dump(gb, 'filename.joblib')
dump(lr, 'filename.joblib')

['filename.joblib']